In [1]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold

In [2]:
df = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_result_stemming.pkl')
dfNo = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_result_nostemming.pkl')

In [4]:
kf = KFold(n_splits=5)
i = 1
for train_index, test_index in kf.split(df):
#     print(train_index[0],"-",train_index[len(train_index)-1])
#     print(test_index[0],"+",test_index[len(test_index)-1])
    X_train = df['text'].iloc[train_index[0]:train_index[len(train_index)-1]]
    X_test = df['text'].iloc[test_index[0]:test_index[len(test_index)-1]]
    y_train = df['label'].iloc[train_index[0]:train_index[len(train_index)-1]].astype('int')
    y_test = df['label'].iloc[test_index[0]:test_index[len(test_index)-1]].astype('int')
    
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    modelknn2 = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
    modelknn2.fit(X_train,y_train)
    y_pred = modelknn2.predict(X_test)
    result = modelknn2.score(X_test, y_test)
    print("Fold ke-",i,"=",result)
    i+=1

Fold ke- 1 = 0.6901408450704225
Fold ke- 2 = 0.8450704225352113
Fold ke- 3 = 0.7746478873239436
Fold ke- 4 = 0.8028169014084507
Fold ke- 5 = 0.5492957746478874


In [6]:
def split(strt,fnsh,df):
    X_test = df['text'].iloc[strt:fnsh]
    y_test = df['label'].iloc[strt:fnsh].astype('int')
    df.drop(df.index[strt:fnsh])
    X_train = df['text']
    y_train = df['label'].astype('int')    
    return X_train, X_test, y_train, y_test

In [16]:
def classifier(df,distance_name,k_neighbor):
    print("Nama distance=",distance_name)
    print("Jumlah K tetangga=",k_neighbor)
    spl = [0,36,72,108,144,180,216,252,288,324,360]
    tempspl = []
    i = 0
    akurasimax=0
    while (i<len(spl)-1):
        if len(spl)-2 != len(tempspl):
            X_train, X_test, y_train, y_test = split(spl[i],spl[i+2],df)
            tempspl.append(spl[i])
    #   ketika array[i+2] == null (looping terakhir)
        elif (len(spl)-len(tempspl)) == 2:
    #       split last and first
            X_test1 = df['text'].iloc[spl[i]:spl[i+1]]
            y_test1 = df['label'].iloc[spl[i]:spl[i+1]].astype('int')
            X_test2 = df['text'].iloc[tempspl[0]:tempspl[1]]
            y_test2 = df['label'].iloc[tempspl[0]:tempspl[1]].astype('int')
    #       concat last and first
            X_test = pd.concat([X_test1, X_test2], axis=0)
            y_test = pd.concat([y_test1, y_test2], axis=0)
    #       split except last and first
            X_train = df['text'].iloc[tempspl[1]:tempspl[len(tempspl)-1]]
            y_train = df['label'].iloc[tempspl[1]:tempspl[len(tempspl)-1]].astype('int')
        vectorizer = TfidfVectorizer()
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)

        modelknn = KNeighborsClassifier(n_neighbors=k_neighbor, weights='uniform', metric=distance_name)
        modelknn.fit(X_train,y_train)
        y_pred = modelknn.predict(X_test)
        result = modelknn.score(X_test, y_test)
        print("Fold ke-",i+1,"=",result)
        if (result >= akurasimax):
            akurasimax = result
            foldke = i+1
        i+=1
    print("Akurasi terbaik adalah",akurasimax,"pada fold ke-",foldke)
    print("------------------------------------")

In [17]:
%%time
dfcount = [df,dfNo]
distancecount = ['euclidean','manhattan','minkowski']
kcount = [1,3,5,7,9,11,13,15]
for x in range(2):
    for y in range(len(distancecount)):
        for z in range(len(kcount)):
            if x==0:
                print('Dengan Stemming')
            else:
                print('Tanpa Stemming')
            classifier(dfcount[x],distancecount[y],kcount[z])

Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 1
Fold ke- 1 = 1.0
Fold ke- 2 = 1.0
Fold ke- 3 = 1.0
Fold ke- 4 = 1.0
Fold ke- 5 = 1.0
Fold ke- 6 = 1.0
Fold ke- 7 = 1.0
Fold ke- 8 = 1.0
Fold ke- 9 = 1.0
Fold ke- 10 = 0.5555555555555556
Akurasi terbaik adalah 1.0 pada fold ke- 9
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 3
Fold ke- 1 = 0.8611111111111112
Fold ke- 2 = 0.875
Fold ke- 3 = 0.8333333333333334
Fold ke- 4 = 0.7916666666666666
Fold ke- 5 = 0.7777777777777778
Fold ke- 6 = 0.8194444444444444
Fold ke- 7 = 0.8333333333333334
Fold ke- 8 = 0.8194444444444444
Fold ke- 9 = 0.8472222222222222
Fold ke- 10 = 0.5277777777777778
Akurasi terbaik adalah 0.875 pada fold ke- 2
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 5
Fold ke- 1 = 0.8472222222222222
Fold ke- 2 = 0.875
Fold ke- 3 = 0.8472222222222222
Fold ke- 4 = 0.7777777777777778
Fold ke- 5 = 0.75
Fold ke- 6 = 0.7916666666

Fold ke- 1 = 0.8194444444444444
Fold ke- 2 = 0.8194444444444444
Fold ke- 3 = 0.7916666666666666
Fold ke- 4 = 0.8055555555555556
Fold ke- 5 = 0.8194444444444444
Fold ke- 6 = 0.8333333333333334
Fold ke- 7 = 0.8055555555555556
Fold ke- 8 = 0.7361111111111112
Fold ke- 9 = 0.6944444444444444
Fold ke- 10 = 0.5416666666666666
Akurasi terbaik adalah 0.8333333333333334 pada fold ke- 6
------------------------------------
Dengan Stemming
Nama distance= minkowski
Jumlah K tetangga= 9
Fold ke- 1 = 0.7916666666666666
Fold ke- 2 = 0.7916666666666666
Fold ke- 3 = 0.7916666666666666
Fold ke- 4 = 0.8055555555555556
Fold ke- 5 = 0.7916666666666666
Fold ke- 6 = 0.8194444444444444
Fold ke- 7 = 0.8055555555555556
Fold ke- 8 = 0.7361111111111112
Fold ke- 9 = 0.6944444444444444
Fold ke- 10 = 0.6111111111111112
Akurasi terbaik adalah 0.8194444444444444 pada fold ke- 6
------------------------------------
Dengan Stemming
Nama distance= minkowski
Jumlah K tetangga= 11
Fold ke- 1 = 0.7361111111111112
Fold ke- 2 

Fold ke- 5 = 0.20833333333333334
Fold ke- 6 = 0.20833333333333334
Fold ke- 7 = 0.2638888888888889
Fold ke- 8 = 0.2777777777777778
Fold ke- 9 = 0.19444444444444445
Fold ke- 10 = 0.125
Akurasi terbaik adalah 0.2777777777777778 pada fold ke- 8
------------------------------------
Tanpa Stemming
Nama distance= manhattan
Jumlah K tetangga= 13
Fold ke- 1 = 0.18055555555555555
Fold ke- 2 = 0.20833333333333334
Fold ke- 3 = 0.20833333333333334
Fold ke- 4 = 0.2222222222222222
Fold ke- 5 = 0.2361111111111111
Fold ke- 6 = 0.2222222222222222
Fold ke- 7 = 0.25
Fold ke- 8 = 0.2361111111111111
Fold ke- 9 = 0.18055555555555555
Fold ke- 10 = 0.125
Akurasi terbaik adalah 0.25 pada fold ke- 7
------------------------------------
Tanpa Stemming
Nama distance= manhattan
Jumlah K tetangga= 15
Fold ke- 1 = 0.20833333333333334
Fold ke- 2 = 0.25
Fold ke- 3 = 0.2361111111111111
Fold ke- 4 = 0.2222222222222222
Fold ke- 5 = 0.2638888888888889
Fold ke- 6 = 0.2638888888888889
Fold ke- 7 = 0.2916666666666667
Fold ke-

# try--

In [ ]:
X_train = df['text'].iloc[72:]
X_test = df['text'].iloc[:72]
y_train = df['label'].iloc[72:].astype('int')
y_test = df['label'].iloc[:72].astype('int')

In [ ]:
X = df['text']
y = df['label'].astype('int')
# X = dfNo['text']
# y = dfNo['label'].astype('int')

In [ ]:
# Pembentukan tf-idf untuk pembobotan
vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(X)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [ ]:
%%time
modelknn2 = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='manhattan')
modelknn2.fit(X_train,y_train)

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_nostemming_173_splitmanual.sav'
joblib.dump(modelknn2, filename)

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_stemming.sav'
loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, y_test)
# print(result)

In [ ]:
y_pred = modelknn2.predict(X_test)
y_pred

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred

In [ ]:
modelknn2.predict_proba(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
result = modelknn2.score(X_test, y_test)
print(result)

In [ ]:
result_class = pd.DataFrame({'Actual Result':y_test, 'Predicted Result':y_pred})
result_class

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result_class)